This demo shows how to use the `RayDist` repository by 

1. generating the avalanche dataset for Speck 32/64
2. exploring the computational resources
3. running an ensemble neural network analysis

Please execute the following cells by pressing 

`Shift + Enter`

# Install Requirements 

The following modules are required: 

In [4]:
%cat requirements.txt

toml
ray
tensorflow
watchdog

scipy == 1.7.3

Install them by running the following cell:

In [ ]:
!pip install -r requirements.txt

# Choose a round to analyze 

In [5]:
# Here we will analyze round 7 of Speck 32/64.
ROUND_IDs = [6]

# Generate Data 

Create a folder for the data

In [8]:
!mkdir 'speck_32_64'

Generate `number_of_samples = 300_000` avalanche units

In [10]:
#### CONFIGURATION ######

number_of_samples = 300_000

#########################

from avalanche_data_generator.speck_32_64 import speck_k64_p32_o32_r22 as data_generator
import numpy as np

dataset=data_generator.generate_avalanche_dataset(int(number_of_samples))

for r in ROUND_IDs:
    np.save(f"speck_32_64/round{r}_sequences300k.npy", dataset[ROUND_ID])

# Have a look at the available Resources 

The Nvidia A100-DGX server provides 

* `4 A100-SXM GPUs, each with 40'536 MiB memory (and in addition a graphical GPU)` 
* `128 CPU cores`

In comparison, the free Google Colaboratory provides

* `1 Tesla T4 GPU with 15'109 MiB memory`
* `2 CPU cores`

The available resources can be explored by using for example the `ray.available_resources()` or `nvidia-smi`:

In [1]:
import ray
ray.init()
ray.available_resources()

{'GPU': 5.0,
 'object_store_memory': 94416284467.0,
 'memory': 210304663757.0,
 'CPU': 128.0,
 'node:172.31.102.80': 1.0}

In [2]:
!nvidia-smi

Sat Jun  4 10:54:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   36C    P0    51W / 275W |      3MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:47:00.0 Off |                    0 |
| N/A   

# Run `RayDist` neural network analysis 

In [ ]:
import toml
from raydist.filemanager import FileManager

for r in ROUND_IDs:
    
    savepath = f'demo_speck32_roundid{r}'
    !mkdir '{savepath}'

    F = FileManager(savepath)
    
    datapath = f'speck_32_64/round{r}_sequences300k.npy'
    
    # the meaning of each parameter is explained in detail in 
    # `raydist/example_config.cfg`
    cfgdict = {'datapath': datapath,
                 'N_BITS': 1024,
                 'N_GPUS': 4,
                 'N_ACTORS_PER_GPU': 6,
                 'NUM_GPUS': 0.15,
                 'NUM_CPUS': 5,
                 'MODEL_ID': 'gohr_generalized',
                 'MODEL_STRENGTH': 1,
                 'MODEL_INPUTS': 1024,
                 'MODEL_OUTPUTS': 63,
                 'N_EPOCHS': 5,
                 'N_TRAIN': 147456,
                 'N_VAL': 145512,
                 'BATCHSIZE': 4096,
                 'EARLY_STOPPING': True,
                 'MAKE_NEW_FILTERS': True,
                 'N_FILTERS': 100,
                 'N_INPUT_FILTER_ELEMENTS': 961,
                 'FILTER_STRATEGY': 'random',
                 'TARGET_BITS': [],
                 'SAVE_WEIGHTS': False,
                 'DATA_STRATEGY': 'zero'}
                 
    with open(F.filename_cfg(), 'w') as configfile:
        toml.dump(cfgdict, configfile)
        
    print("="*len(datapath))
    print(datapath)
        
    !python -m raydist.run --savepath '{savepath}'

speck_32_64/round6_sequences300k.npy
||           time            |   NN finished   | pred. bits ||  best bit  |  acc (%)   |   n pred   |  p value   ||
||   2022-06-04_11h10m35s    |      0/100      |   0/1024   ||    nan     |    nan     |    nan     |    nan     ||
||   2022-06-04_11h11m18s    |      1/100      |  63/1024   ||    448     |   50.280   |     1      |   0.033    ||
||   2022-06-04_11h11m18s    |      2/100      |  125/1024  ||    711     |   50.403   |     1      |   0.0021   ||
||   2022-06-04_11h11m19s    |      3/100      |  180/1024  ||    244     |   50.403   |     1      |   0.0021   ||
||   2022-06-04_11h11m19s    |      4/100      |  232/1024  ||    244     |   50.403   |     1      |   0.0021   ||
||   2022-06-04_11h11m20s    |      5/100      |  289/1024  ||    244     |   50.403   |     1      |   0.0021   ||
||   2022-06-04_11h11m20s    |      7/100      |  338/1024  ||    244     |   50.403   |     1      |   0.0021   ||
||   2022-06-04_11h11m20s    |     